# Generation of LCA indicators and associated .mod and .dat files

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import bw2data as bd
from mescal import *
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing

In [3]:
ei_version = '3.10.1'
year = 2020 # 2020 or 2050
with_regioinvent = True

In [4]:
path_inputs = '../01_Notebooks/Data/'
path_model = '../02_Model/'
path_data = f'../02_Model/{year}/'
if with_regioinvent:
    path_data_lca = path_data + 'with_regioinvent/'
    path_results = f'../03_Results/LCA/{year}/with_regioinvent/'
else:
    path_data_lca = path_data + 'without_regioinvent/'
    path_results = f'../03_Results/LCA/{year}/without_regioinvent/'

## Initialize the EnergyScope model

In [5]:
# AMPL licence 
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL licence file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [6]:
# Define the solver options
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}

In [7]:
with open(path_model + 'QC_objective_function.mod', 'w') as f:
        f.write('minimize obj: TotalCost;')

In [8]:
# Initialize the 2021 QC model with .mod and .dat files
model = Model(
    mod_files=[
        path_model+'QC_es_main.mod',
        path_model+'QC_objective_function.mod',
    ],
    dat_files=[
        path_data+'QC_data.dat',
        path_data+'QC_mob_techs_dist_B2D.dat',
        path_data+'QC_techs_B2D.dat',
        path_data+'QC_mob_params.dat',
    ],
)

In [9]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [10]:
# Solve the model and get results
results = postprocessing(es.calc())

Error:
	../02_Model/2020/QC_mob_techs_dist_B2D.dat
	line 464 offset 67311
		invalid subscript layers_in_out['PLANE_FREIGHT_BIO_ELD','CO2_E']

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>



AMPLException: ../02_Model/2020/QC_mob_techs_dist_B2D.dat
line 464 offset 67311
	invalid subscript layers_in_out['PLANE_FREIGHT_BIO_ELD','CO2_E']

## Load data and initialize the ESM class

In [11]:
# Load the data
mapping = pd.read_csv(path_inputs+f'mapping.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')  # open file and press enter in one computation cell to avoid misreading
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
tech_specifics = pd.read_csv(path_inputs+'technology_specifics.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')

In [12]:
# Load the model from energyscope model
model = results.parameters['layers_in_out'].reset_index().rename(columns={'index0':'Name', 'index1':'Flow', 'layers_in_out':'Amount'}).drop(columns=['Run'])
model = model[model['Amount'] != 0]

In [13]:
# Databases names
name_main_database = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_{year}+truck_carculator regionalized_75'
name_regioinvent_db = f'regioinvent_cutoff_{ei_version}_image_SSP2-Base_{year}+truck_carculator_75'
name_biosphere_db = f'ecoinvent-{ei_version}-biosphere'
name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
name_es_database = f'EnergyScope_CA-QC_{year}_75'

if not with_regioinvent:
    name_main_database += '_wo_regioinvent'
    name_es_database += '_wo_regioinvent'

In [14]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version}')

In [15]:
if with_regioinvent:
    main_db = Database(db_names=[name_main_database, name_regioinvent_db], create_pickle=True)
else:
    main_db = Database(db_names=name_main_database, create_pickle=True)

Loaded ecoinvent_cutoff_3.10.1_image_SSP2-Base_2020+truck_carculator regionalized from pickle!
Loaded regioinvent_cutoff_3.10.1_image_SSP2-Base_2020+truck_carculator from pickle!


In [16]:
ranking_best_ecoinvent_locations_for_QC = [
    'CA-QC', # Quebec
    'CA', # Canada
    'CAN', # Canada in IMAGE
    'CAZ', # Canada - Australia - New Zealand in REMIND
    'RNA', # North America
    'US', # United States
    'USA', # United States in REMIND and IMAGE
    'GLO', # Global average 
    'RoW', # Rest of the world
]

In [17]:
# Add CPC categories to the main database
main_db.add_CPC_categories()

In [18]:
# Change the main database name if needed
if mapping['Database'].iloc[0] != name_main_database:
    mapping['Database'] = len(mapping) * [name_main_database]
missing_flows = main_db.test_mapping_file(mapping)

Mapping successfully linked to the database


In [19]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    main_database=main_db,
    esm_db_name=name_es_database,
    
    # Optional inputs
    technology_compositions=techno_compositions,
    tech_specifics=tech_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=True,
    accepted_locations=['CA-QC'],
    locations_ranking=ranking_best_ecoinvent_locations_for_QC,
    esm_location='CA-QC',
    results_path_file=path_results,
    biosphere_db_name=name_biosphere_db,
    
    # If we want regionalized results 
    spatialized_biosphere_db=Database(name_spatialized_biosphere_db),
)

Getting activity data


100%|██████████| 110559/110559 [00:01<00:00, 96702.45it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 110559/110559 [00:00<00:00, 1136864.80it/s]


Loaded biosphere3_spatialized_flows from brightway!


In [20]:
esm.check_inputs()

List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value.

--> ['CO2_E', 'DEC_RENOVATION', 'DHN_RENOVATION', 'DIESEL_S', 'ELEC_S', 'GASOLINE_S', 'HT_LT', 'LT_DEC_WH', 'LT_DHN_WH', 'NG_S', 'RES_GEO', 'RES_HYDRO', 'RES_SOLAR', 'RES_TIDAL', 'RES_WIND_OFFSHORE', 'RES_WIND_ONSHORE', 'STO_CO2', 'STO_DIE', 'STO_ELEC', 'STO_GASO', 'STO_H2', 'STO_NG']

List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow).

--> ['BATTERY', 'DEC_TH_STORAGE', 'DHN_TH_STORAGE', 'EHV_GRID', 'HV_GRID', 'LV_GRID', 'MV_GRID']

List of technologies that are in the tech_specifics file but not in the mapping file (this will not be a problem in the workflow).

--> ['AEC_OG', 'MOB_FREIGHT_ELD', 'MOB_FREIGHT_LD', 'MOB_FREIGHT_MD', 'MOB_FREIGHT_SD', 'MOB_PRIVATE_ELD', 'MOB_PRIVATE_LD', 'MOB_PRIVATE_MD', 'MOB_PRIVATE_SD', 'MOB_PUBLIC_ELD', 'MOB_PUBLIC_LD', 'MOB_P

In [21]:
# Adapt mapping file to ESM location
esm.change_location_mapping_file()
esm.mapping.to_csv(path_inputs+f'mapping.csv', index=False)

In [24]:
main_db = {}  # Free memory

### Generate ESM database

In [22]:
# Foreground regionalization, double-counting removal, and efficiency harmonization
esm.create_esm_database()

### Starting to add construction and resource activities database ###
No location found in your ranking for waste polystyrene - market group for waste polystyrene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyvinylchloride - market group for waste polyvinylchloride
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste plastic, mixture - market group for waste plastic, mixture
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyethylene - market group for waste polyethylene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste glass - market group for waste glass
--> Have to keep the initial location: RER
No location found in your ranking for iridium - platinum group metal, extraction and refinery operations
--> Have to keep the initial location: ZA

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


Title: Writing activities to SQLite3 database:
  Started: 03/05/2025 13:17:20
  Finished: 03/05/2025 13:18:06
  Total time elapsed: 00:00:45
  CPU %: 58.60
  Memory %: 36.08
EnergyScope_CA-QC_2020 written to Brightway!
### Database written ###
--> Time: 346.0 seconds


In [23]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(path_results+'mapping_with_new_codes.csv', index=False)

## Generate LCA metrics 

In [22]:
# If the ESM database is already created
esm.mapping = pd.read_csv(path_results+'mapping_with_new_codes.csv')

In [23]:
methods = ['IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10']

### Life-cycle emissions

In [ ]:
# Lifetime harmonization
R_long = esm.compute_impact_scores(
    methods=methods,
    specific_lcia_abbrev=['m_CCS', 'TTHH', 'TTEQ'],
    impact_abbrev=impact_abbrev,
)

Getting activity data


100%|██████████| 1332/1332 [00:00<00:00, 86939.40it/s]


Adding exchange data to activities


100%|██████████| 33071/33071 [00:03<00:00, 9546.96it/s] 


Filling out exchange data


100%|██████████| 1332/1332 [00:02<00:00, 462.18it/s]


Loaded EnergyScope_CA-QC_2020 from brightway!


In [ ]:
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

### Direct emissions

In [28]:
activities_subject_to_double_counting = pd.read_csv(f'{path_results}activities_subject_to_double_counting.csv')

In [29]:
R_long_direct_emissions = esm.compute_impact_scores(
    methods=methods,
    specific_lcia_abbrev=['m_CCS', 'TTHH', 'TTEQ'],
    assessment_type='direct emissions',
    impact_abbrev=impact_abbrev,
    activities_subject_to_double_counting=activities_subject_to_double_counting,
    overwrite=False,
)

Getting activity data


100%|██████████| 1335/1335 [00:00<00:00, 47519.36it/s]


Adding exchange data to activities


100%|██████████| 33420/33420 [00:03<00:00, 10811.11it/s]


Filling out exchange data


100%|██████████| 1335/1335 [00:02<00:00, 587.56it/s] 


Loaded EnergyScope_CA-QC_2050 from brightway!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 02/27/2025 12:02:14
  Finished: 02/27/2025 12:02:16
  Total time elapsed: 00:00:01
  CPU %: 86.00
  Memory %: 3.73
EnergyScope_CA-QC_2050_direct_emissions written to Brightway!


In [30]:
R_long_direct_emissions.to_csv(f'{path_results}impact_scores_direct_emissions.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

## Create the .mod and .dat files

In [31]:
# To skip the impact assessment step
R_long = pd.read_csv(f'{path_results}impact_scores.csv')
R_long_direct_emissions = pd.read_csv(f'{path_results}impact_scores_direct_emissions.csv')

In [32]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': year,
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [33]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    impact_abbrev=impact_abbrev,
    path=path_data_lca,
    metadata=metadata,
    file_name='QC_techs_lca',
)

In [34]:
# Create .dat file for direct emissions only
esm.normalize_lca_metrics(
    assessment_type='direct emissions',
    R=R_long_direct_emissions,
    max_per_cat=pd.read_csv(path_data_lca+'QC_techs_lca_max.csv'),
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    impact_abbrev=impact_abbrev,
    path=path_data_lca,
    metadata=metadata,
    file_name='QC_techs_lca_direct',
)

In [35]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    path=path_data_lca,
    metadata=metadata,
    file_name='QC_objectives_lca',
)

In [36]:
# Create the .mod file for direct emissions only
esm.generate_mod_file_ampl(
    assessment_type='direct emissions',
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    path=path_data_lca,
    metadata=metadata,
    file_name='QC_objectives_lca_direct',
)